In [1]:
import pandas as pd
import numpy as np
import torch
import random
from sentence_transformers import SentenceTransformer, util


In [2]:
# !conda install -c conda-forge ipywidgets


In [3]:
pre_data = pd.read_csv("../data/total_data.csv")
print(pre_data.shape)


(127051, 8)


In [4]:
#remove duplicates by url, reindex
data = pre_data.drop_duplicates(subset='url', keep="first") #keep first or last
data.index = np.arange(len(data))


print(len(pre_data),len(data)) 


127051 94805


In [5]:
data

,title,description,url,data_type,source,ori_label,ori_source,label
0,충청남도 당진시_산업단지 입주기업 현황_20220304,"충청남도 당진시 산업단지 입주 기업 현황입니다.(연번, 단지명, 회사명, 업종명, ...",https://www.data.go.kr/data/15106898/fileData.do,텍스트,충청남도 당진시,산업·통상·중소기업,공공데이터포털,제조소비
1,전라남도 강진군_미곡처리장현황_20221006,"전라남도 강진군 미곡처리장 현황에 대한 데이터로소 시군, 명칭, 소재지, 관리부서,...",https://www.data.go.kr/data/15106993/fileData.do,텍스트,전라남도 강진군,농림,공공데이터포털,식품건강
2,경상남도 거창군_월별 코로나 확진자 현황_20221007,경상남도 거창군 월별 코로나19 관련 확진자 및 사망자 수 현황 데이터로 월별 확진...,https://www.data.go.kr/data/15098865/fileData.do,텍스트,경상남도 거창군,보건,공공데이터포털,보건의료
3,충청남도 부여군_1인당지방세부담액_20221006,상기 데이터는 연도별 주민 1인당 또는 세대 당 부담된 지방세액에 대한 정보로 조...,https://www.data.go.kr/data/15080007/fileData.do,텍스트,충청남도 부여군,재정·세제·금융,공공데이터포털,경제금융
4,전라남도 보성군_담배소매인지정현황_20221006,"전라남도 보성군 담배소매인지정현황에 관한 공공데이터로 업소명, 지번주소, 도로명주소...",https://www.data.go.kr/data/15035564/fileData.do,텍스트,전라남도 보성군,일반공공행정,공공데이터포털,행정법률
...,...,...,...,...,...,...,...,...
94800,서울시 구로구 식품첨가물제조업 인허가 정보,구로구의 식품을 제조 또는 가공할 때 기호가치를 향상시키거나 영양가치를 높일목적의 ...,https://data.seoul.go.kr/dataList/OA-18293/S/1...,"SHEET,OpenAPI",구로구 스마트도시정책관 빅데이터담당관,보건,서울데이터광장,보건의료
94801,서울시 송파구 국제회의시설업 인허가 정보,송파구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설...,https://data.seoul.go.kr/dataList/OA-16990/S/1...,"SHEET,OpenAPI",송파구 스마트도시정책관 빅데이터담당관,문화/관광,서울데이터광장,문화관광
94802,서울시 중구 국제회의시설업 인허가 정보,중구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설을...,https://data.seoul.go.kr/dataList/OA-16968/S/1...,"SHEET,OpenAPI",중구 스마트도시정책관 빅데이터담당관,문화/관광,서울데이터광장,문화관광
94803,서울시 강남구 국제회의시설업 인허가 정보,강남구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설...,https://data.seoul.go.kr/dataList/OA-16989/S/1...,"SHEET,OpenAPI",강남구 스마트도시정책관 빅데이터담당관,문화/관광,서울데이터광장,문화관광


In [6]:
#load description sbert model
sents = data['description']
#[print(sents[i], "\n") for i in range(5)]
sbert = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

### Sample 

In [7]:
sample = sbert.encode(sents[:5])
len(sample[0])

768

In [8]:
snubert_sample_tensor = torch.tensor(sample)
torch.save(snubert_sample_tensor, '../data/snubert_sample_tensor.pt')


In [9]:
#selecting tensor values (if needed)
snubert_sample_tensor.index_select(0, torch.tensor(np.arange(3)))

tensor([[-0.6379, -0.4988, -0.4288,  ...,  0.6085,  0.3506,  0.7135],
        [ 0.3231,  0.2048, -0.2758,  ...,  0.5903,  0.4753, -0.3940],
        [ 0.4405, -0.6636,  0.3967,  ..., -0.2820, -0.8777,  0.2145]])

In [10]:
index_rand = 2
similarity_sample = util.cos_sim(sample[index_rand] , snubert_sample_tensor).sort(descending=True)
similarity_sample

torch.return_types.sort(
values=tensor([[1.0000, 0.4430, 0.4085, 0.3700, 0.3513]]),
indices=tensor([[2, 3, 1, 0, 4]]))

In [11]:
similarity_sample.values[0][1:5]

tensor([0.4430, 0.4085, 0.3700, 0.3513])

### Actual Data

In [12]:
index_rand = 2

#make and save tensor
snubert_des = sbert.encode(sents)
snubert_des_tensor = torch.tensor(snubert_des)
torch.save(snubert_des_tensor, '../data/snubert_des_tensor_2.pt')




In [13]:
similarity = util.cos_sim(snubert_des[index_rand] , snubert_des_tensor).sort(descending=True)
similarity

torch.return_types.sort(
values=tensor([[ 1.0000,  0.7405,  0.7179,  ..., -0.0459, -0.0498, -0.0547]]),
indices=tensor([[    2,  7638, 19255,  ..., 42052, 55205, 74683]]))

In [14]:
len(similarity.values[0])
id_ls = similarity.indices[0].tolist() #[1,k+1]

In [15]:
data.iloc[index_rand]['title']

'경상남도 거창군_월별 코로나 확진자 현황_20221007'

In [16]:
id_ls[1:k+1]

[7638, 19255, 21080, 5286, 19140]

In [17]:
for id in id_ls[1:k+1] :
    print(data.iloc[id]['title'])

경상남도 창녕군_코로나19 현황_20220823
경상남도 함양군_코로나19 확진자 및 사망자 현황_20220228
경상남도 양산시_코로나19 확진자 및 사망자 데이터_20220204
경상남도 밀양시_코로나19 감염자 및 사망자 수_20220905
경상남도 남해군_코로나확진자_20220331


In [18]:
for id in id_ls[1:11] :
    print(data.iloc[id]['title'])

경상남도 창녕군_코로나19 현황_20220823
경상남도 함양군_코로나19 확진자 및 사망자 현황_20220228
경상남도 양산시_코로나19 확진자 및 사망자 데이터_20220204
경상남도 밀양시_코로나19 감염자 및 사망자 수_20220905
경상남도 남해군_코로나확진자_20220331
전라북도 순창군_코로나 19 확진자 및 사망자 현황_20220401
경상남도_거창군_행정마을별세대인구수_20220630
경상남도 거창군_관광객수_20220228
경상남도 통영시_코로나19 확진자 및 사망자 데이터_20221005
경상남도 진주시_코로나19 확진자수 현황_20220228


In [30]:
data.iloc[77401]['title']

'대전광역시_환경 분쟁 조정 현황'

In [28]:
# 중복 데이터 관련
# https://github.com/Paul-scpark/AI-dev-course-prj/issues/24
sim_dupe = util.cos_sim(snubert_des[77401], snubert_des_tensor).sort(descending=True)
sim_dupe
id_ls = sim_dupe.indices[0].tolist() #[1,k+1]

for id in id_ls[1:11] :
    print(data.iloc[id]['title'])

환경부 화학물질안전원_화학 사고 정보_20220531
충청남도_민원현황_20220731
울산시설공단_민원처리현황_20220831
한국남동발전㈜_환경관련 민원사항 현황_12/31/2020
대전광역시_국민신문고 접수 처리현황(민원분야)_20220731
환경부 화학물질안전원_화학사고정보
대전광역시 동구_대기배출시설 사업장 현황_20220803
대전광역시 유성구_대기배출사업장_20220405
서울시 환경분쟁조정 사건 진행 현황
서울특별시_환경분쟁조정 사건 진행 현황


In [31]:
print(data.iloc[10101]['title'])


경찰청 경상남도경찰청_자동차 운전면허 학원 현황_20220630


In [39]:
n,k = len(data), 5 #n=94805

#inspect top 5 
for idx in random.sample(range(n),k) :
    similarity = util.cos_sim(snubert_des[idx] , snubert_des_tensor).sort(descending=True)
    id_ls = similarity.indices[0].tolist() #[1,k+1]

    print("Recommendations for : ", data.iloc[idx]['title'],'\n',data.iloc[idx]['description'])

    for id in id_ls[1:10] :
        print(data.iloc[id]['title'], '\n',data.iloc[id]['description'])
    print("===============")

Recommendations for :  인천광역시 서구_공기 조화 제조업체_20220919 
 인천광역시 서구 공기 조화 제조업체 현황 (회사명, 생산품, 업종명, 대표주소, 전화번호, 팩스번호 등) 에 관한 데이터입니다.
인천광역시 서구_제조업체_20220901 
 인천광역시 서구 제조업체 현황 (회사명, 생산품, 업종명, 대표주소, 전화번호, 팩스번호 등) 에 관한 데이터입니다.
인천광역시 서구_승강기 제조업체_20220919 
 인천광역시 서구 승강기 제조업체 현황 (회사명, 생산품, 업종명, 대표주소, 전화번호, 팩스번호 등) 에 관한 데이터입니다.
인천광역시 서구_골판지 제조업체_20220919 
 인천광역시 서구 골판지 제조업체 현황 (회사명, 생산품, 업종명, 대표주소, 전화번호, 팩스번호 등) 에 관한 데이터입니다.
인천광역시 서구_기체 여과기 제조업체_20220919 
 인천광역시 서구 기체 여과기 제조업체 현황 (회사명, 생산품, 업종명, 대표주소, 전화번호, 팩스번호 등) 에 관한 데이터입니다.
인천광역시 서구_가구 제조업체_20220919 
 인천광역시 서구 가구 제조업체 현황 (회사명, 생산품, 업종명, 대표주소, 전화번호, 팩스번호 등) 에 관한 데이터입니다.
인천광역시 서구_간판 및 광고물 제조업체_20220919 
 인천광역시 서구 간판 및 광고물 제조업체 현황 (회사명, 생산품, 업종명, 대표주소, 전화번호, 팩스번호 등) 에 관한 데이터입니다.
인천광역시 서구_근무복 및 작업복 제조업체_20220919 
 인천광역시 서구 근무복 및 작업복 제조업체 현황 (회사명, 생산품, 업종명, 대표주소, 전화번호, 팩스번호 등) 에 관한 데이터입니다.
인천광역시 서구_구조용 금속판 제조업체_20220919 
 인천광역시 서구 구조용 금속판 제조업체 현황 (회사명, 생산품, 업종명, 대표주소, 전화번호, 팩스번호 등) 에 관한 데이터입니다.
인천광역시 서구_도금업 제조업체_20220919 
 인천광역시 서구 도금업 제조업체 현황 (회사명, 

In [ ]:
#